In [1]:
import random
import time
import numpy as np
import math 
import cv2
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO 
from typing import Callable
import os
import carla

2024-05-09 09:32:44.472462: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-09 09:32:44.544958: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-09 09:32:44.873959: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gp-coopperc/anaconda3/envs/Tensor/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cud

In [2]:
SECONDS_PER_EPISODE = 25

N_CHANNELS = 3
HEIGHT = 240
WIDTH = 320


SHOW_PREVIEW = True


SPIN = 10 #angle of random spin

HEIGHT_REQUIRED_PORTION = 0.5 #bottom share, e.g. 0.1 is take lowest 10% of rows
WIDTH_REQUIRED_PORTION = 0.9

In [3]:
class CarEnv(gym.Env):
    SHOW_CAM = SHOW_PREVIEW
    STEER_AMT = 1.0
    im_width = WIDTH
    im_height = HEIGHT
    front_camera = None
    CAMERA_POS_Z = 1.3 
    CAMERA_POS_X = 1.4
    PREFERRED_SPEED = 60
    SPEED_THRESHOLD = 2
    
    def __init__(self):
        super(CarEnv, self).__init__()
        
        self.action_space = spaces.Discrete(9)
        
         # for cropping images
        self.height_from = int(HEIGHT * (1 -HEIGHT_REQUIRED_PORTION))
        self.width_from = int((WIDTH - WIDTH * WIDTH_REQUIRED_PORTION) / 2)
        self.width_to = self.width_from + int(WIDTH_REQUIRED_PORTION * WIDTH)
        self.new_height = HEIGHT - self.height_from
        self.new_width = self.width_to - self.width_from

        self.observation_space = spaces.Box(low=0.0, high=255.0,
                                            shape=(self.new_height, self.new_width, N_CHANNELS), dtype=np.uint8)
        
        self.client = carla.Client("localhost", 2000)
        self.client.set_timeout(4.0)
        self.client.load_world('Town01')
        self.world = self.client.get_world()

        self.settings = self.world.get_settings()
        #self.settings.no_rendering_mode = True
    
        self.world.apply_settings(self.settings)
        self.blueprint_library = self.world.get_blueprint_library()
        self.model_3 = self.blueprint_library.filter('*mini*')[0]
        self.actor_list = []
        
    def maintain_speed(self,s):
        
        if s >= self.PREFERRED_SPEED:
            return 0
        elif s < self.PREFERRED_SPEED - self.SPEED_THRESHOLD:
            return 0.7 # think of it as % of "full gas"
        else:
            return 0.3 # tweak this if the car is way over or under preferred speed 

        
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.cleanup()
        self.collision_hist = []
        self.actor_list = []
        self.lane_invade_hist = []

        self.transform = random.choice(self.world.get_map().get_spawn_points())
        self.point_a = self.transform.location
        self.point_b = carla.Location(x=107.175446, y=129.416809, z=0.003263)  # Destination point

        
    
        print("Trying to spawn vehicle...")
        self.vehicle = self.world.try_spawn_actor(self.model_3, self.transform)
        if self.vehicle is None:
            print("Failed to spawn vehicle! Resetting environment...")
            self.cleanup()  # Clean up any existing actors
            return self.reset()  # Retry resetting the environment
        
        print("Vehicle spawned successfully!")
        self.actor_list.append(self.vehicle)
        self.initial_location = self.vehicle.get_location()

        self.sem_cam = self.blueprint_library.find('sensor.camera.semantic_segmentation')
        self.sem_cam.set_attribute("image_size_x", f"{self.im_width}")
        self.sem_cam.set_attribute("image_size_y", f"{self.im_height}")
        self.sem_cam.set_attribute("fov", f"90")


        camera_init_trans = carla.Transform(carla.Location(z=self.CAMERA_POS_Z,x=self.CAMERA_POS_X))
        self.sensor = self.world.spawn_actor(self.sem_cam, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.sensor)
        self.sensor.listen(lambda data: self.process_img(data))

        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        time.sleep(2) #not detect a collision when the car spawns/falls from sky.
        
        # now apply random yaw so the RL does not guess to go straight
        angle_adj = random.randrange(-SPIN, SPIN, 1)
        trans = self.vehicle.get_transform()
        trans.rotation.yaw = trans.rotation.yaw + angle_adj
        self.vehicle.set_transform(trans)
        
        # showing camera at the spawn point
        if self.SHOW_CAM:
            cv2.namedWindow('Sem Camera',cv2.WINDOW_AUTOSIZE)
            cv2.imshow('Sem Camera', self.front_camera)
            cv2.waitKey(1)
        
        lanesensor = self.blueprint_library.find("sensor.other.lane_invasion")
        self.lanesensor = self.world.spawn_actor(lanesensor, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.lanesensor)
        self.lanesensor.listen(lambda event: self.lane_data(event))
        
        colsensor = self.blueprint_library.find("sensor.other.collision")
        self.colsensor = self.world.spawn_actor(colsensor, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.colsensor)
        self.colsensor.listen(lambda event: self.collision_data(event))
     

        while self.front_camera is None:
            time.sleep(0.01)#Just in case car takes any longer, because we need to be certain the car is done falling from the sky on spawn.

        self.episode_start = time.time()
        self.steering_lock = False
        self.steering_lock_start = None # this is to count time in steering lock and start penalising for long time in steering lock
        
        self.step_counter = 0
        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        
        observation = self.front_camera[self.height_from:,self.width_from:self.width_to]

        return observation.astype(np.uint8), {}
    
    def step(self, action):
        self.step_counter +=1
        steer = action
        # map steering actions
        if steer ==0:
            steer = - 0.9
        elif steer ==1:
            steer = -0.25
        elif steer ==2:
            steer = -0.1
        elif steer ==3:
            steer = -0.05
        elif steer ==4:
            steer = 0.0 
        elif steer ==5:
            steer = 0.05
        elif steer ==6:
            steer = 0.1
        elif steer ==7:
            steer = 0.25
        elif steer ==8:
            steer = 0.9


        v = self.vehicle.get_velocity()
        kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))

        estimated_throttle = self.maintain_speed(kmh)

        self.vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer, brake = 0.0))
        
        distance_to_destination = self.vehicle.get_location().distance(self.point_b)
        distance = self.point_a.distance(self.point_b)

        # storing camera to return at the end in case the clean-up function destroys it
        cam = self.front_camera
        # showing image
        if self.SHOW_CAM:
            cv2.imshow('Sem Camera', cam)
            cv2.waitKey(1)
            
        # track steering lock duration to prevent "chasing its tail"
        lock_duration = 0
        if self.steering_lock == False:
            if steer<-0.6 or steer>0.6:
                self.steering_lock = True
                self.steering_lock_start = time.time()
        else:
            if steer<-0.6 or steer>0.6:
                lock_duration = time.time() - self.steering_lock_start

        # start defining reward from each step
        reward = 0
        done = False
         #punish for collision
        if len(self.collision_hist) != 0:
            done = True
            reward = reward - 300
            self.reset()
            print('COLLISION')
        if len(self.lane_invade_hist) != 0:
            reward = reward - 200
        # punish for steer lock up
        if lock_duration>3:
            reward = reward - 150
            done = True
            self.reset()
            print('LOCK')

        elif lock_duration > 1:
            reward = reward - 20

        # Calculate reward based on distance traveled towards point B    
        if distance_to_destination < distance/4:  # threshold for reaching destination
            reward = reward + 30  # Positive reward for reaching destination
            done = True  # End episode when destination is reached
        elif distance_to_destination < distance/3:
            reward = reward + 20  
        elif distance_to_destination < distance/2:
            reward = reward + 10  
        elif distance_to_destination < distance:
            reward = reward + 5
        else:
            reward = reward - 10  # Negative reward for not reaching destination
            done = False

        # check for episode duration
        if self.episode_start + SECONDS_PER_EPISODE < time.time():
            done = True
            self.cleanup()
            #print(self.actor_list)
            
        observation = cam[self.height_from:,self.width_from:self.width_to]

        return observation.astype(np.uint8), reward, done, False, {}
        
    def cleanup(self):
        for actor in self.actor_list:
            if actor.is_alive:
                actor.destroy()
        #self.actor_list = []
        cv2.destroyAllWindows()

        
    def process_img(self, image):
        image.convert(carla.ColorConverter.CityScapesPalette)
        i = np.array(image.raw_data)
        i = i.reshape((self.im_height, self.im_width, 4))[:, :, :3] # this is to ignore the 4th Alpha channel - up to 3
        self.front_camera = i/255.0

    def collision_data(self, event):
        self.collision_hist.append(event)
        
    def lane_data(self, event):
        self.lane_invade_hist.append(event)

In [4]:
models_dir = f"models/{int(time.time())}/"
logdir = f"logs/{int(time.time())}/"

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):
    os.makedirs(logdir)


env = CarEnv()

env.reset()

model = PPO('CnnPolicy', env, verbose=1,learning_rate=0.001,tensorboard_log=logdir)

TIMESTEPS = 500_000 # individual steps
iters = 0
while iters<4:  # how many training iterations you want
    iters += 1
    print('Iteration ', iters,' is to start...')
    try:
        model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=f"PPO")
        print('Iteration', iters, 'has been trained')
        model.save(f"{models_dir}/{TIMESTEPS*iters}")
    except ValueError as e:
        print(f"Error during training iteration {iters}: {e}")
        continue

Trying to spawn vehicle...
Vehicle spawned successfully!
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Iteration  1  is to start...
Trying to spawn vehicle...
Vehicle spawned successfully!
Logging to logs/1715206789/PPO_0
Trying to spawn vehicle...
Vehicle spawned successfully!
LOCK
-----------------------------
| time/              |      |
|    fps             | 269  |
|    iterations      | 1    |
|    time_elapsed    | 7    |
|    total_timesteps | 2048 |
-----------------------------
Trying to spawn vehicle...
Vehicle spawned successfully!
COLLISION
Trying to spawn vehicle...
Vehicle spawned successfully!
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.05e+03    |
|    ep_rew_mean          | -1.79e+04   |
| time/                   |             |
|    fps                  | 232         |
|    iterations           | 2           

LOCK
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.7e+03     |
|    ep_rew_mean          | -4.78e+04   |
| time/                   |             |
|    fps                  | 183         |
|    iterations           | 9           |
|    time_elapsed         | 100         |
|    total_timesteps      | 18432       |
| train/                  |             |
|    approx_kl            | 0.006109534 |
|    clip_fraction        | 0.036       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.17       |
|    explained_variance   | 0           |
|    learning_rate        | 0.001       |
|    loss                 | 1.69e+03    |
|    n_updates            | 80          |
|    policy_gradient_loss | -0.00889    |
|    value_loss           | 4.18e+03    |
-----------------------------------------
Trying to spawn vehicle...
Vehicle spawned successfully!
COLLISION
Trying to spawn vehicle...
Vehicle spawned suc

Trying to spawn vehicle...
Vehicle spawned successfully!
LOCK
Trying to spawn vehicle...
Vehicle spawned successfully!
LOCK
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 2.51e+03      |
|    ep_rew_mean          | -4.29e+04     |
| time/                   |               |
|    fps                  | 180           |
|    iterations           | 17            |
|    time_elapsed         | 193           |
|    total_timesteps      | 34816         |
| train/                  |               |
|    approx_kl            | 0.00085678266 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -2.11         |
|    explained_variance   | 1.19e-07      |
|    learning_rate        | 0.001         |
|    loss                 | 2.35e+03      |
|    n_updates            | 160           |
|    policy_gradient_loss | -0.00362      |
|    value_loss           | 4.95e+03    

Trying to spawn vehicle...
Vehicle spawned successfully!
LOCK
Trying to spawn vehicle...
Vehicle spawned successfully!
LOCK
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.33e+03    |
|    ep_rew_mean          | -4.24e+04   |
| time/                   |             |
|    fps                  | 175         |
|    iterations           | 25          |
|    time_elapsed         | 292         |
|    total_timesteps      | 51200       |
| train/                  |             |
|    approx_kl            | 0.001745384 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.06       |
|    explained_variance   | 0           |
|    learning_rate        | 0.001       |
|    loss                 | 2.89e+03    |
|    n_updates            | 240         |
|    policy_gradient_loss | -0.00386    |
|    value_loss           | 5.91e+03    |
------------------------------------

Trying to spawn vehicle...
Vehicle spawned successfully!
COLLISION
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
LOCK
Trying to spawn vehicle...
Vehicle spawned successfully!
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 2.25e+03      |
|    ep_rew_mean          | -5.38e+04     |
| time/                   |               |
|    fps                  | 174           |
|    iterations           | 33            |
|    time_elapsed         | 386           |
|    total_timesteps      | 67584         |
| train/                  |               |
|    approx_kl            | 4.9967086e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -2.02         |
|    explained_variance   | 0             |
|    learning_rate        | 0.001         |
|    loss                 | 1.03e+06      |
|    n_up

...

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 3.79e+03      |
|    ep_rew_mean          | -1.34e+05     |
| time/                   |               |
|    fps                  | 214           |
|    iterations           | 210           |
|    time_elapsed         | 2007          |
|    total_timesteps      | 1935360       |
| train/                  |               |
|    approx_kl            | 2.1752203e-07 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.8          |
|    explained_variance   | -1.19e-07     |
|    learning_rate        | 0.001         |
|    loss                 | 2.9e+06       |
|    n_updates            | 9440          |
|    policy_gradient_loss | 5.79e-06      |
|    value_loss           | 5e+06         |
-------------------------------------------
Trying to spawn vehicle...
Vehicle spawned successfully!
COLLISION
Trying to

Trying to spawn vehicle...
Vehicle spawned successfully!
COLLISION
Trying to spawn vehicle...
Vehicle spawned successfully!
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 3.69e+03      |
|    ep_rew_mean          | -1.33e+05     |
| time/                   |               |
|    fps                  | 213           |
|    iterations           | 218           |
|    time_elapsed         | 2092          |
|    total_timesteps      | 1951744       |
| train/                  |               |
|    approx_kl            | 1.1167198e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.81         |
|    explained_variance   | 0             |
|    learning_rate        | 0.001         |
|    loss                 | 2.01e+06      |
|    n_updates            | 9520          |
|    policy_gradient_loss | -6.72e-05     |
|    value_loss           | 4.24e+06    

Trying to spawn vehicle...
Vehicle spawned successfully!
COLLISION
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 3.6e+03       |
|    ep_rew_mean          | -1.31e+05     |
| time/                   |               |
|    fps                  | 213           |
|    iterations           | 227           |
|    time_elapsed         | 2177          |
|    total_timesteps      | 1970176       |
| train/                  |               |
|    approx_kl            | 0.00079846766 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.78         |
|    explained_variance   | 0             |
|    learning_rate        | 0.001         |
|    loss                 | 8.61e+03      |
|    n_updates            | 9610          |
|    policy_gradient_loss | -0.000699     |
|    value_loss           | 1.76e+04      |
-------------------------------------------
Trying to

Trying to spawn vehicle...
Vehicle spawned successfully!
COLLISION
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 3.67e+03      |
|    ep_rew_mean          | -1.35e+05     |
| time/                   |               |
|    fps                  | 213           |
|    iterations           | 235           |
|    time_elapsed         | 2257          |
|    total_timesteps      | 1986560       |
| train/                  |               |
|    approx_kl            | 3.8380356e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.78         |
|    explained_variance   | 0             |
|    learning_rate        | 0.001         |
|    loss                 | 1.93e+06      |
|    n_updates            | 9690          |
|    policy_gradient_loss | -0.000148     |
|    value_loss           | 2.41e+06      |
-------------------------------------------
Trying to

COLLISION
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 3.65e+03      |
|    ep_rew_mean          | -1.33e+05     |
| time/                   |               |
|    fps                  | 213           |
|    iterations           | 243           |
|    time_elapsed         | 2334          |
|    total_timesteps      | 2002944       |
| train/                  |               |
|    approx_kl            | 0.00014152261 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.78         |
|    explained_variance   | 0             |
|    learning_rate        | 0.001         |
|    loss                 | 7.19e+05      |
|    n_updates            | 9770          |
|    policy_gradient_loss | -0.000177     |
|    value_loss           | 1.79e+06      |
-------------------------------------------
Trying to spawn vehicle...
Vehicle spawned successfully!
COLLISION

In [4]:
env = CarEnv()
env.reset()

models_dir = "/home/gp-coopperc/CARLA_TEST/models/1715206789/"
model_path = f"{models_dir}/2000000.zip"
model = PPO.load(model_path, env=env)

Trying to spawn vehicle...
Vehicle spawned successfully!
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [5]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor


env = Monitor(env) 
evaluate_policy(model, env, n_eval_episodes=10,deterministic=True, return_episode_rewards= True)

/home/gp-coopperc/anaconda3/envs/Tensor/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.world to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.world` for environment variables or `env.get_wrapper_attr('world')` that will search the reminding wrappers.
  logger.warn(


Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
COLLISION
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
COLLISION
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!


KeyboardInterrupt: 